## Use gQuant to Build cuStreamz Examples

The stream processing market is experiencing exponential growth with businesses relying heavily on real-time analytics, inferencing, monitoring, and more. [cuStreamz](https://medium.com/rapids-ai/gpu-accelerated-stream-processing-with-rapids-f2b725696a61) is the first GPU-accelerated streaming data processing library. Written in Python, it is built on top of RAPIDS, the GPU-accelerator for data science libraries. cuStreamz accelerates [Streamz](https://streamz.readthedocs.io/en/latest/) by leveraging RAPIDS cuDF under the hood to perform computations on streaming data in an accelerated manner on GPUs. The Python library Streamz helps you build pipelines to manage continuous streams of data. 

It is easy to wrap the cuStreamz streaming pipelines into gQuant nodes and visualize it. In this notebook example, we will step-by-step build a stream pipeline example in gQuant. You will learn how to write a basic gQuant node, register it for the gQuant Jupyterlab plugin, use GPU to accelerate the computations.

Let's first import the streamz libraries and gQuant libraries.

In [1]:
import streamz
import streamz.dataframe
import cudf

In [2]:
from gquant.dataframe_flow import Node
from gquant.dataframe_flow import NodePorts, PortsSpecSchema, TaskSpecSchema
from gquant.dataframe_flow import ConfSchema
from gquant.dataframe_flow import TaskGraph

## Double the Streaming Numbers by Streamz 

A gQuant node is defined by inheriting from the `Node` class and overriding methods `init`, `columns_setup`, `ports_setup`, `conf_schema`, `process`.

The `ports_setup` must return an instance of `NodePorts` which encapsulates the ports specs. Ports specs are dictionaries with port attributes/options per `PortsSpecSchema`.

In the case of the `PointNode` below the input port is an empty dictionary, since no inputs are required, and the output has two ports `points_df_out` and `points_ddf_out`. It can output two types of dataframe frames depends who connects it.

The `process` method receives a input dictionary where keys are input ports and values are input data. It return a dictionary where the keys correspond to the output ports. 

The `columns_setup` is used to compute the output columns information. It returns a dictionary where keys correspond to the output ports.

The `conf_schema` is used to define the Node configuration [JSON schema](https://json-schema.org/). gQuantlab UI uses [RJSF](https://github.com/rjsf-team/react-jsonschema-form) project to generate HTML form elements based on the JSON schema. [RJSF playground](https://rjsf-team.github.io/react-jsonschema-form/)  is a good place to learn how to write JSON schema and visualize it. `conf_schema` returns `ConfSchema` which encapsulate the JSON schema and UI schema together.

The `column` and `port_types` information sometimes are determined dynamically. gQuant provides a few utility functions to help get dynamical graph information. `self.get_connected_inports()` will return a dictionay where keys are connected inport names and values are inport types. 
`self.get_input_columns()` will return a dictionary where keys are connected inport names and values are column name/type paris from the parent node. `self.outport_connected(port_name)` method returns a boolean if the output port `port_name` is connected.

As a `hello-world` example for streamz, we will build an example that doubles continuous streams of numbers and print it. We need 3 nodes for it: source, map stream and sink. 

### Define the Source Stream Node

In [3]:
class StreamNode(Node):

    def ports_setup(self):
        input_ports = {}
        output_ports = {
            'stream_out': {
                PortsSpecSchema.port_type: streamz.Stream
            },
        }
        return NodePorts(inports=input_ports, outports=output_ports)

    def conf_schema(self):
        json = {
            "title": "Source node configure",
            "type": "object",
            "properties": {
            }
        }
        ui = {}
        return ConfSchema(json=json, ui=ui)

    def init(self):
        self.required = {}

    def columns_setup(self):
        columns_out = {
            'stream_out': {'element': 'number'
            },
        }
        return columns_out

    def process(self, inputs):
        output = {}
        output.update({'stream_out': streamz.Stream()})
        return output


As shown above, this source node has one output port which outputs `streamz.Stream` object. It has no configuration, so we just leave it empty. In the `columns_setup` method, we specify that the element type in the stream is a number. The down-stream node can connect to it only if its `self.required` has the same key value pair.

In the `process` method, it outputs the `stream.Stream()` as the source end of the pipeline. Later, we will use the `emit` method to add numbers to it.

### Define the Double Element Node

In [4]:
class TransformNode(Node):

    def ports_setup(self):
        input_ports = {
             'stream_in': {
                PortsSpecSchema.port_type: streamz.Stream
            },           
        }
        output_ports = {
            'stream_out': {
                PortsSpecSchema.port_type: streamz.Stream
            },
        }
        return NodePorts(inports=input_ports, outports=output_ports)

    def conf_schema(self):
        json = {
            "title": "Transform Node configure",
            "type": "object",
            "properties": {
            }
        }
        ui = {}
        return ConfSchema(json=json, ui=ui)

    def init(self):
        self.required = {}

    def columns_setup(self):
        self.required = {
            'stream_in': {'element': 'number'}
        }
        columns_out = {
            'stream_out': {'element': 'number'
            },
        }
        return columns_out

    def process(self, inputs):
        in_stream = inputs['stream_in']
        output = {}
        def double(ele):
            return ele*2
        output.update({'stream_out': in_stream.map(double)})
        return output


`TransformNode` definition is similar to `StreamNode` but it has an input port `stream_in`. It defines the key value pair `element->number` in the `self.required` dictionary, so it is compatible to connect to the source node. 

In the `process` method, it maps each elements in the stream by `double` function.

### Define the Print Elements Node

In [5]:
class SinkNode(Node):

    def ports_setup(self):
        input_ports = {
             'stream_in': {
                PortsSpecSchema.port_type: streamz.Stream
            },           
        }
        output_ports = {
            'stream_out': {
                PortsSpecSchema.port_type: streamz.Stream
            },
        }
        return NodePorts(inports=input_ports, outports=output_ports)

    def conf_schema(self):
        json = {
            "title": "Simple SinkNode configure",
            "type": "object",
            "properties": {
            }
        }
        ui = {}
        return ConfSchema(json=json, ui=ui)

    def init(self):
        self.required = {}

    def columns_setup(self):
        self.required = {
            'stream_in': {'element': 'number'}
        }
        columns_out = {
            'stream_out': {'element': 'number'
            },
        }
        return columns_out

    def process(self, inputs):
        in_stream = inputs['stream_in']
        output = {}
        output.update({'stream_out': in_stream.sink(print)})
        return output

The `SinkNode` is very simple, it just adds `print` function to print each of the elements in the sink. 

### Define the Graph Programmatically 

Having these three nodes, we can construct a simple task graph to connect them. We added a `Output_Collector` node so that we can get a handle to both the source and sink. 

In [6]:
module_name = 'streamz'
source_spec = {
    TaskSpecSchema.task_id: 'source',
    TaskSpecSchema.node_type: StreamNode,
    TaskSpecSchema.conf: {},
    TaskSpecSchema.inputs: {},
    TaskSpecSchema.module: module_name
}


task_spec = {
    TaskSpecSchema.task_id: 'double',
    TaskSpecSchema.node_type: TransformNode,
    TaskSpecSchema.conf: {},
    TaskSpecSchema.inputs: {
        'stream_in': 'source.stream_out'
    },
    TaskSpecSchema.module: module_name
}

sink_spec = {
    TaskSpecSchema.task_id: 'sink',
    TaskSpecSchema.node_type: SinkNode,
    TaskSpecSchema.conf: {},
    TaskSpecSchema.inputs: {
        'stream_in': 'double.stream_out'
    },
    TaskSpecSchema.module: module_name
}

out_spec = {
    TaskSpecSchema.task_id: '',
    TaskSpecSchema.node_type: "Output_Collector",
    TaskSpecSchema.conf: {},
    TaskSpecSchema.inputs: {
        'in0': 'sink.stream_out',
        'in1': 'source.stream_out'
    }
}

task_list = [source_spec, task_spec, sink_spec, out_spec]

### Dynamically Register the GQuant Nodes

The task graph defined above is ready to be evaluated programmatically by 

In [7]:
task_graph = TaskGraph(task_list)
task_graph.run()

Results([('sink.stream_out', <sink: print>), ('source.stream_out', <Stream>)])

The result has both the source and sink nodes stored as a dictionary.

To use the task graph in the gQuant Jupyterlab plugin, we can either put them in a python file and edit `gquantrc` file to register them. You can find an example in the `05_customize_nodes_with_ports` notebook. Or, we can dynamically register them by `register_lab_node` method.

In [8]:
TaskGraph.register_lab_node(module_name, StreamNode)
TaskGraph.register_lab_node(module_name, TransformNode)
TaskGraph.register_lab_node(module_name, SinkNode)

GQuantWidget(sub=HBox())

We can visualize this simple task graph

In [9]:
task_graph = TaskGraph(task_list)
task_graph.draw()

GQuantWidget(sub=HBox(), value=[OrderedDict([('id', 'source'), ('type', 'StreamNode'), ('conf', {}), ('inputs'…

Let evaluate it and get the source node so we can add numbers to it by calling `emit` method.

In [10]:
r = task_graph.run()

The result is saved to variable `r`. It can be used as a dictionary. Let's find the keys:

In [11]:
r.get_keys()

('sink.stream_out', 'source.stream_out')

We can emit some numbers to the source and see the stream processing is in effect.

In [12]:
for i in range(10):
    r['source.stream_out'].emit(i)

0
2
4
6
8
10
12
14
16
18


## Show the Doubling Results in a Sliding Window

Imagine you want to build a monitor that can watch the latest 50 numbers in the stream. Streamz library provides a few stream process nodes that make it easy to do. Let's add a sliding window node that can group the numbers in a sliding window of defined length. 

### Define the Sliding Window Node

In [13]:
class SlideWindowNode(Node):

    def ports_setup(self):
        input_ports = {
             'stream_in': {
                PortsSpecSchema.port_type: streamz.Stream
            },           
        }
        output_ports = {
            'stream_out': {
                PortsSpecSchema.port_type: streamz.Stream
            },
        }
        return NodePorts(inports=input_ports, outports=output_ports)

    def conf_schema(self):
        json = {
            "title": "SlideWindow Node configure",
            "type": "object",
            "properties": {
                "window": {
                    "type": "integer"
                }
            }
        }
        ui = {}
        return ConfSchema(json=json, ui=ui)

    def init(self):
        self.required = {}

    def columns_setup(self):
        self.required = {
            'stream_in': {'element': 'number'}
        }
        columns_out = {
            'stream_out': {'element': 'numbers'
            },
        }
        return columns_out

    def process(self, inputs):
        in_stream = inputs['stream_in']
        output = {}
        output.update({'stream_out': in_stream.sliding_window(self.conf['window'], return_partial=False)})
        return output

It is similar to the `TransformNode`. In the `process` method, it uses `in_stream.sliding_window` method to group the elemnts in a sliding window. In the `conf_schema`, we define a `window` field of type `integer`. It defines the sliding window size. As it aggregate the numbers in a window, the `columns_out` has element type `numbers` instead of `number`. Note, the key-value string can be arbitary strings as long as the compatible input/output ports share the same key-value pairs. 

### Define the Plot Node

Define a node to visualize the numbers online

In [14]:
import bqplot
import numpy as np
import bqplot.pyplot as plt

class PlotSinkNode(Node):

    def ports_setup(self):
        input_ports = {
             'stream_in': {
                PortsSpecSchema.port_type: streamz.Stream
            },           
        }
        output_ports = {
            'fig_out': {
                PortsSpecSchema.port_type: bqplot.Figure
            },
        }
        return NodePorts(inports=input_ports, outports=output_ports)

    def conf_schema(self):
        json = {
            "title": "Plot configure",
            "type": "object",
            "properties": {
            }
        }
        ui = {}
        return ConfSchema(json=json, ui=ui)

    def init(self):
        self.required = {}

    def columns_setup(self):
        self.required = {
            'stream_in': {'element': 'numbers'}
        }
        columns_out = {
            'fig_out': {}
        }
        return columns_out

    def process(self, inputs):
        in_stream = inputs['stream_in']
        axes_options = {'x': {'label': 'x'}, 'y': {'label': 'y'}}
        x = []
        y = []
        fig = plt.figure(animation_duration=10)
        lines = plt.plot(x=x, y=y, colors=['red', 'green'], axes_options=axes_options)
        output = {}
        def update(numbers):
            if isinstance(numbers[0], tuple):
                elements  = len(numbers[0])
            else:
                elements = len(numbers)
            x = np.arange(elements)
            if isinstance(numbers, cudf.Series):
                y = numbers.to_array()
            else:
                y = np.array(numbers)
            lines.x = x
            lines.y = y
        in_stream.sink(update)
        output.update({'fig_out': fig})
        return output

This `PlotSinkNode` is a sink of the stream that use `update` function to plot the numbers in the sliding window. So we can see an animiation of number changes as new numbers are added.

Register these two new nodes:

In [15]:
TaskGraph.register_lab_node(module_name, PlotSinkNode)
TaskGraph.register_lab_node(module_name, SlideWindowNode)

We already wired the graph in the `plot.gq.yaml` file. Load it from the disk:

In [16]:
task_graph = TaskGraph.load_taskgraph('../taskgraphs/streamz/plot.gq.yaml')
task_graph.draw()

GQuantWidget(sub=HBox(), value=[OrderedDict([('id', 'source'), ('type', 'StreamNode'), ('conf', {}), ('inputs'…

It applies `sliding_windw` after `doubling` the numbers, then feed them to `plot` sink node.

Run it to get the source and sink

In [17]:
r = task_graph.run()

Show the plot:

In [18]:
r['plot.fig_out']

Figure(animation_duration=10, axes=[Axis(label='x', scale=LinearScale(), side='bottom'), Axis(label='y', orien…

Let's add `sine` wave numbers to the stream:

In [19]:
for i in range(1000):
    r['source.stream_out'].emit(np.sin(i/3.14))

You can see the moving sin waves in the plot. Try to change the sliding window size and play with it!

## Process Two Branches of the stream

Streamz library can handle complicated streams in the pipeline. For illustration purpose, we add a `ZipNode` that can merge two streams together so the element from the two streams are zipped into a tuple.

### Define the Zip Node to Combine Branches

In [20]:
class ZipNode(Node):

    def ports_setup(self):
        input_ports = {
             'stream1_in': {
                PortsSpecSchema.port_type: streamz.Stream
            }, 
             'stream2_in': {
                PortsSpecSchema.port_type: streamz.Stream
            },       
        }
        output_ports = {
            'stream_out': {
                PortsSpecSchema.port_type: streamz.Stream
            },
        }
        return NodePorts(inports=input_ports, outports=output_ports)

    def conf_schema(self):
        json = {
            "title": "Zip Node configure",
            "type": "object",
            "properties": {
            }
        }
        ui = {}
        return ConfSchema(json=json, ui=ui)

    def init(self):
        self.required = {}

    def columns_setup(self):
        self.required = {
            'stream1_in': {'element': 'numbers'},
            'stream2_in': {'element': 'numbers'}
        }
        columns_out = {
            'stream_out': {'element': 'numbers'
            },
        }
        return columns_out

    def process(self, inputs):
        in_stream1 = inputs['stream1_in']
        in_stream2 = inputs['stream2_in']
        output = {}
        output.update({'stream_out': in_stream1.zip(in_stream2)})
        return output

`ZipNode` defines two input ports: `stream1_in` and `stream2_in`. In the `process` method, it calls `in_stream1.zip(in_stream2)` to zip them together.

Register this node:

In [21]:
TaskGraph.register_lab_node(module_name, ZipNode)

We wired the graph in the `two_branches.gq.yaml` file. Load it from the disk:

In [22]:
task_graph = TaskGraph.load_taskgraph('../taskgraphs/streamz/two_branches.gq.yaml')
task_graph.draw()

GQuantWidget(sub=HBox(), value=[OrderedDict([('id', 'source'), ('type', 'StreamNode'), ('conf', {}), ('inputs'…

It first doubles the elements. Then it branches off. In one branch, it doubles the elements again. After merging these two branches by `ZipNode`, it sends the stream of tuples to the `plot` sink node. 

Let's run this graph:

In [23]:
r = task_graph.run()

Show the plot:

In [24]:
r['plot.fig_out']

Figure(animation_duration=10, axes=[Axis(label='x', scale=LinearScale(), side='bottom'), Axis(label='y', orien…

Let's add sin wave numbers to the stream

In [25]:
for i in range(1000):
    r['source.stream_out'].emit(np.sin(i/3.14))

It shows two sine waves. One is doubled and the other one is quadrupled

## Use GPU to Accelerate

Stream processing can be well accelerated in the GPU by RAPIDS cudf dataframe. Streamz can work with both Pandas and cuDF dataframes, to provide sensible streaming operations on continuous tabular data.

We will take the above streaming pipeline and convert it to be accelerated in the GPU.

### Define the Node to Convert a Tuple of Numbers to Cudf Dataframe

The sliding window node aggregate the numbers in a tuple. We can define a Node to convert the tuple of numbers to cudf dataframe

In [26]:
class TupleToCudf(Node):

    def ports_setup(self):
        input_ports = {
             'stream_in': {
                PortsSpecSchema.port_type: streamz.Stream
            },           
        }
        output_ports = {
            'stream_out': {
                PortsSpecSchema.port_type: streamz.Stream
            },
        }
        return NodePorts(inports=input_ports, outports=output_ports)

    def conf_schema(self):
        json = {
            "title": "Tuple of data to Cudf dataframe Node configure",
            "type": "object",
            "properties": {
            }
        }
        ui = {}
        return ConfSchema(json=json, ui=ui)

    def init(self):
        self.required = {}

    def columns_setup(self):
        self.required = {
            'stream_in': {'element': 'numbers'}
        }
        columns_out = {
            'stream_out': {'element': 'cudf'
            },
        }
        return columns_out

    def process(self, inputs):
        in_stream = inputs['stream_in']
        output = {}
        def convert(ele):
            df = cudf.DataFrame({'x': ele})
            return df
        output.update({'stream_out': in_stream.map(convert)})
        return output

It maps tuple of numbers to cudf dataframe elements by `in_stream.map` method.

### Define the Node to Convert Cudf stream to streamz.DataFrame 

The `streamz.dataframe` module provides a streaming dataframe object that implements many of dataframe methods. It provides a Pandas-like interface on streaming data. Let's write a `ToDataFrame` Node that converts the cudf dataframe stream to streaming dataframe.  

In [27]:
# Create a streamz dataframe to get stateful word count
class ToDataFrame(Node):

    def ports_setup(self):
        input_ports = {
             'stream_in': {
                PortsSpecSchema.port_type: streamz.Stream
            },           
        }
        output_ports = {
            'df_out': {
                PortsSpecSchema.port_type: streamz.dataframe.DataFrame
            },
        }
        return NodePorts(inports=input_ports, outports=output_ports)

    def conf_schema(self):
        json = {
            "title": "streamz Dataframe Node configure",
            "type": "object",
            "properties": {
            }
        }
        ui = {}
        return ConfSchema(json=json, ui=ui)

    def init(self):
        self.required = {}

    def columns_setup(self):
        self.required = {
            'stream_in': {'element': 'cudf'}
        }
        columns_out = {
            'df_out': {'x': 'float64'
            },
        }
        return columns_out

    def process(self, inputs):
        in_stream = inputs['stream_in']
        output = {}
        sdf = streamz.dataframe.DataFrame(in_stream, example=cudf.DataFrame({'x':[]}))
        output.update({'df_out': sdf})
        return output

In the `process` method, it calls `treamz.dataframe.DataFrame` constructor to convert a stream of cudf dataframe into a streaz dataframe.

### Define the Node to Double the numbers via streamz.DataFrame API

It is straight forward to write a Node that doubles one column in a dataframe. 

In [28]:
class GPUDouble(Node):

    def ports_setup(self):        
        input_ports = {
             'df_in': {
                PortsSpecSchema.port_type: streamz.dataframe.DataFrame
            },           
        }
        output_ports = {
            'df_out': {
                PortsSpecSchema.port_type: streamz.dataframe.DataFrame
            },
        }
        return NodePorts(inports=input_ports, outports=output_ports)

    def conf_schema(self):
        json = {
            "title": "GPU double Node configure",
            "type": "object",
            "properties": {
            }
        }
        ui = {}
        return ConfSchema(json=json, ui=ui)

    def init(self):
        self.required = {}

    def columns_setup(self):
        self.required = {
            'df_in': {'x': 'float64'}
        }
        columns_out = {
            'df_out': {'x': 'float64'
            },
        }
        return columns_out

    def process(self, inputs):
        in_df = inputs['df_in']
        in_df['x'] = in_df['x'] * 2
        output = {}
        output.update({'df_out': in_df})
        return output

## Define the Node to Convert streamz.DataFrame to Cudf.Series stream

Once the streamz Dataframe finished the computation, we can convert it back to normal stream for display.

In [29]:
class ToStream(Node):

    def ports_setup(self):
        input_ports = {
             'df_in': {
                PortsSpecSchema.port_type: streamz.dataframe.DataFrame
            },           
        }
        output_ports = {
            'stream_out': {
                PortsSpecSchema.port_type: streamz.Stream
            },
        }
        return NodePorts(inports=input_ports, outports=output_ports)

    def conf_schema(self):
        json = {
            "title": "df to stream Node configure",
            "type": "object",
            "properties": {
            }
        }
        ui = {}
        return ConfSchema(json=json, ui=ui)

    def init(self):
        self.required = {}

    def columns_setup(self):
        self.required = {
            'df_in': {'x': 'float64'}
        }
        columns_out = {
            'stream_out': {'element': 'numbers'
            },
        }
        return columns_out

    def process(self, inputs):
        in_df = inputs['df_in']
        output = {}
        output.update({'stream_out': in_df.stream.pluck('x')})
        return output

The stream can be accessed by the `stream` property in the streamz Dataframe. We call `pluck` method to return the `x` column of the dataframe.

Register the newly added Nodes:

In [30]:
TaskGraph.register_lab_node(module_name, TupleToCudf)
TaskGraph.register_lab_node(module_name, ToDataFrame)
TaskGraph.register_lab_node(module_name, GPUDouble)
TaskGraph.register_lab_node(module_name, ToStream)

We already wired the graph in the `plot.gq.yaml` file. Load it from the disk:

In [31]:
task_graph = TaskGraph.load_taskgraph('../taskgraphs/streamz/gpu_double.gq.yaml')
task_graph.draw()

GQuantWidget(sub=HBox(), value=[OrderedDict([('id', 'source'), ('type', 'StreamNode'), ('conf', {}), ('inputs'…

It first aggregates the numbers into a tuple of numbers by sliding window node. It converts the tuple of numbers into a stream of cudf dataframes. After converting it to streamz.Dataframe, we can use normal dataframe API to do transformations on the dataframe. Then it converts back to stream of cudf.Series of numbers for ploting and printing. 

Let's run the TaskGraph:

In [32]:
r = task_graph.run()

Show the plot:

In [33]:
r['plot.fig_out']

Figure(animation_duration=10, axes=[Axis(label='x', scale=LinearScale(), side='bottom'), Axis(label='y', orien…

Let's add `sine` wave numbers to the stream:

In [34]:
for i in range(200):
    r['source.stream_out'].emit(np.sin(i/3.14))

0     0.000000
1     0.626230
2     1.189481
3     1.633106
4     1.912490
5     1.999535
6     1.885488
7     1.581818
8     1.119066
9     0.543769
10   -0.086215
11   -0.707528
12   -1.257685
13   -1.681357
14   -1.935935
15   -1.995818
16   -1.854981
17   -1.527590
18   -1.046570
19   -0.460296
20    0.172269
21    0.787510
22    1.323550
23    1.726482
24    1.955782
25    1.988390
26    1.821026
27    1.470522
28    0.972128
29    0.375968
30   -0.258003
31   -0.866027
32   -1.386955
33   -1.768398
34   -1.971993
35   -1.977266
36   -1.783685
37   -1.410720
38   -0.895880
39   -0.290941
40    0.343258
41    0.942935
42    1.447782
43    1.807026
44    1.984538
45    1.962465
46    1.743028
47    1.348296
48    0.817966
49    0.205373
Name: x, dtype: float64
0     0.626230
1     1.189481
2     1.633106
3     1.912490
4     1.999535
5     1.885488
6     1.581818
7     1.119066
8     0.543769
9    -0.086215
10   -0.707528
11   -1.257685
12   -1.681357
13   -1.935935
14   -1.995818
1

In [35]:
## Clean up

In [36]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}